In [1]:
import os
api_key = os.environ["LANGCHAIN_API"] 

gemini_api = os.environ["GEMINI_API"]

In [2]:
from langchain_google_genai import GoogleGenerativeAI

In [3]:
llm = GoogleGenerativeAI(
    model="gemini-2.5-flash-lite"
)

## Part - 5 Multi Query

In [4]:
#load blog
 
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# split

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)

splits= text_splitter.split_documents(blog_docs)


In [6]:
#index 

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_community.vectorstores import Chroma


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001", 
    api_key=gemini_api,
    credentials= None
)

vectorstore = Chroma.from_documents(documents = splits,
                                   embedding = embeddings)

retriever = vectorstore.as_retriever()

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an AI language model assistant, Your takse is to generate five different versions of the given user questions and to retrieve relevant documents from the database. By generating multiple different perspectives on the user question, your goal is to make the user overcome some of the limitations of the distance based similarity search, provide these alternatives responses separated by newlines. Original question {question}"""

prompt_pers = ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_pers 
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [9]:
from langchain_core.load import dumps, loads

def get_uniq(documents : list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]

In [10]:
#retrieve

question = "what is task decomposition for LLM agents ?"
retrieval_chain = generate_queries | retriever.map() | get_uniq
docs = retrieval_chain.invoke({"question": question})
len(docs)

C:\Users\dhruv\AppData\Local\Temp\ipykernel_12592\4290818311.py:7: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


5

In [11]:
from operator import itemgetter

#RAG 

template = """answer the following question based on this context:
        {context}
        
        Question : {question}"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()

)

In [12]:
final_rag_chain.invoke({"question" : question})

'Task decomposition for LLM agents involves breaking down large, complex tasks into smaller, more manageable subgoals. This process allows the agent to handle intricate tasks more efficiently. This decomposition can be achieved through several methods:\n\n*   **LLM prompting:** Using simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?"\n*   **Task-specific instructions:** Providing explicit instructions tailored to the task, such as "Write a story outline" for novel writing.\n*   **Human inputs:** Incorporating guidance from human users.\n*   **Chain of Thought (CoT):** Instructing the model to "think step by step" to decompose hard tasks into simpler ones.\n*   **Tree of Thoughts (ToT):** Expanding on CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.'

## Part 6 : RAG Fusion

In [13]:
template = """You are an AI language model assistant, Your task is to generate five different versions of the given user questions and to retrieve relevant documents from the database. By generating multiple different perspectives on the user question, your goal is to make the user overcome some of the limitations of the distance based similarity search, provide these alternatives responses separated by newlines. Original question {question}"""

prompt_pers = ChatPromptTemplate.from_template(template)

In [14]:
def reciprocal_rank_fusion(results: list[list], k = 60):
    """reciprocal rank fusion function that takes in multiple lists from ranked documents and an optional paramater k is used in the RRF formula"""

    fused_scores = {}

    for rank, doc in enumerate(docs):
        doc_str = dumps(doc)

        if doc_str not in fused_scores:
            fused_scores[doc_str] = 0
        
        prev_score = fused_scores[doc_str]
        fused_scores[doc_str] += 1 / (rank + k)
    
    reranked_res = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse = True)
    ]

    return reranked_res

ret_chain_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = ret_chain_fusion.invoke({"question": question})
len(docs)

5

In [16]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": ret_chain_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

final_rag_chain.invoke({"question":question})

['Task decomposition for LLM agents refers to the process of breaking down large, complex tasks into smaller, more manageable subgoals. This enables the agent to handle intricate tasks more efficiently. This decomposition can be achieved through various methods, including:',
 '',
 '*   **LLM prompting:** Using simple instructions like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?".',
 '*   **Task-specific instructions:** Providing explicit directives for particular tasks, such as "Write a story outline." for novel writing.',
 '*   **Human inputs:** Receiving direct guidance from a human user.',
 '*   **Chain of Thought (CoT):** Instructing the model to "think step by step" to break down difficult tasks into simpler ones.',
 '*   **Tree of Thoughts (ToT):** Exploring multiple reasoning possibilities at each step, generating several thoughts per step to create a tree structure.']